<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/mcda_py_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/MCDA_UT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THE MCDA PROJECT: Criação da Ferramenta

## Início

#### Vamos ao código!

In [ ]:
'''
  Aqui toda a execução do código se inicia, não se esqueça de executar
  esta célula que é fundamental para o funcionamento do código
'''

# Importar as bibliotecas a serem utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
'''
  DICA: Divida os dados qualitativos em categorias (exemplo na CÉLULA 2),
  aqui está uma tabela para auxiliar no processo.
'''

# - Dividir em 5 categorias
# (0 - 20% (Nada desejável),
# 20 - 40% (Pouco desejável),
# 40 - 60% (Médio desejável),
# 60 - 80% (Bem desejável),
# 80 - 100% (Muito desejável))

'\n  DICA: Divida os dados qualitativos em categorias (exemplo na CÉLULA 2),\n  aqui está uma tabela para auxiliar no processo.\n'

# Realizando a montagem dos gráficos

## **CÉLULA 1 - DADOS NUMÉRICOS**

In [ ]:
import pandas as pd
import plotly.express as px

# Leitura do arquivo CSV com a codificação ISO-8859-1
teste_df = pd.read_csv('base_expandida.csv', encoding='ISO-8859-1')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
teste_df = teste_df.drop([0, 1]).reset_index(drop=True)

# Separando os valores 'BOM' e 'NEUTRO'
valores_bom = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')

# Convertendo apenas as colunas numéricas para float
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom[col] = float(valores_bom[col])
        valores_neutro[col] = float(valores_neutro[col])
        teste_df[col] = teste_df[col].astype(float)

# Função para calcular a pontuação baseada na proximidade ao neutro e ao bom
def calcular_pontuacao(valor, bom, neutro, peso):
    if bom - neutro == 0:
        return 0
    elif valor >= neutro:
        return ((valor - neutro) / (bom - neutro)) * peso
    else:
        return -((neutro - valor) / neutro) * peso

# Aplicando a função de pontuação para cada atributo numérico
for col in montante_relacional.index:
    if tipos_dados[col] == 'number':
        teste_df[col + '_pontuacao'] = teste_df.apply(lambda x: calcular_pontuacao(x[col], valores_bom[col], valores_neutro[col], montante_relacional[col]), axis=1)

# Somando as pontuações para obter a pontuação total
teste_df['Pontuacao_Total'] = teste_df[[col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] == 'number']].sum(axis=1)

# Adicionando classificadores 'BOM' e 'NEUTRO' ao DataFrame
teste_df.loc[teste_df['Modelo'] == 'BOM', 'Color'] = 'Classificadores'
teste_df.loc[teste_df['Modelo'] == 'NEUTRO', 'Color'] = 'Classificadores'
teste_df.loc[~teste_df['Modelo'].isin(['BOM', 'NEUTRO']), 'Color'] = 'Desempenho'

# Ordenando os produtos pela pontuação total
teste_df = teste_df.sort_values(by='Pontuacao_Total', ascending=False).reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados
pontuacao_maxima = teste_df[teste_df['Color'] == 'Desempenho']['Pontuacao_Total'].max()
produtos_recomendados = teste_df[(teste_df['Color'] == 'Desempenho') & (teste_df['Pontuacao_Total'] == pontuacao_maxima)]

# Exibindo todos os produtos recomendados com detalhes para leigos
for _, produto in produtos_recomendados.iterrows():
    print(f"Produto recomendado: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
    print("Pontos de vantagem e justificativas:")

    pontuacoes_detalhadas = []
    for col in montante_relacional.index:
        if tipos_dados[col] == 'number':
            pontuacao_atributo = produto[col + '_pontuacao']
            if pontuacao_atributo > 0:
                # Contexto adicional para leigos
                justificativa = (
                    f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                    f"em comparação com o mínimo aceitável ('NEUTRO')."
                )
                pontuacoes_detalhadas.append(justificativa)

    # Exibindo as justificativas detalhadas para o produto
    for detalhe in pontuacoes_detalhadas:
        print(detalhe)
    print("\n")

# Preparando os dados para o gráfico
df_analise = teste_df.copy()

# Definindo cores baseadas nas pontuações
# df_analise['Color'] = df_analise['Pontuacao_Total'].apply(
#     lambda x: 'Classificadores' if x >= 0 else 'Desempenho'
# )

# Definindo as cores com base na pontuação total usando lambda
df_analise['Color'] = df_analise.apply(
    lambda row: 'Classificadores' if row['Modelo'] in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if row['Pontuacao_Total'] >= 0 else 'Pontuação Negativa'),
    axis=1
)

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',       # Para BOM e NEUTRO
    'Pontuação Positiva': 'Pontuação positiva',  # Para Desempenho
    'Pontuação Negativa': 'Pontuação negativa'   # Para pontuação negativa
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Plotando o gráfico
graph = px.bar(df_analise, x='Pontuacao_Total', y='Modelo', orientation='h',
               color='Color', text_auto=False, hover_name='Color',
               hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
               color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'total ascending'})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise),
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()


Produto recomendado: Clevo com pontuação total de 5.45
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').


Produto recomendado: Eurocom com pontuação total de 5.45
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').




In [62]:
import pandas as pd
import plotly.express as px
import numpy as np

# Leitura do arquivo CSV com a codificação UTF-8
teste_df = pd.read_csv('base_new_2.csv', encoding='UTF-8', sep=';')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
proporcionalidade = teste_df.iloc[2].drop('Modelo')

# Separando os DataFrames de 'BOM', 'NEUTRO' e os demais para cálculo
valores_bom_df = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro_df = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')
calculo_df = teste_df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

# Convertendo os tipos de dados conforme especificado
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom_df[col] = float(valores_bom_df[col])
        valores_neutro_df[col] = float(valores_neutro_df[col])
        calculo_df[col] = calculo_df[col].astype(float)
    elif tipos_dados[col] == 'boolean':
        valores_bom_df[col] = valores_bom_df[col] == 'TRUE'
        valores_neutro_df[col] = valores_neutro_df[col] == 'TRUE'
        calculo_df[col] = calculo_df[col].map({'TRUE': True, 'FALSE': False})

# Função para calcular a pontuação baseada na proximidade ao neutro e ao bom
def calcular_pontuacao(valor, bom, neutro, peso, proporcionalidade_tipo):
    if bom == neutro:
        return 0
    elif proporcionalidade_tipo == 'proportional':
        if isinstance(valor, bool):
            valor_numeric = 1.0 if valor == bom else 0.0
            bom_numeric = 1.0 if bom else 0.0
            neutro_numeric = 1.0 if neutro else 0.0
            if bom_numeric == neutro_numeric:
                return 0
            elif valor_numeric >= neutro_numeric:
                return ((valor_numeric - neutro_numeric) / (bom_numeric - neutro_numeric)) * peso
            else:
                return -((neutro_numeric - valor_numeric) / neutro_numeric) * peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            bom_numeric = 1.0 if bom else 0.0
            neutro_numeric = 1.0 if neutro else 0.0
            if bom_numeric == neutro_numeric:
                return 0
            elif valor >= neutro_numeric:
                return ((valor - neutro_numeric) / (bom_numeric - neutro_numeric)) * peso
            else:
                return -((neutro_numeric - valor) / neutro_numeric) * peso
        elif valor >= neutro:
            return ((valor - neutro) / (bom - neutro)) * peso
        else:
            return -((neutro - valor) / neutro) * peso
    elif proporcionalidade_tipo == 'i_proportional':
        if isinstance(valor, bool):
            valor_numeric = 1.0 if valor == bom else 0.0
            bom_numeric = 1.0 if bom else 0.0
            neutro_numeric = 1.0 if neutro else 0.0
            if bom_numeric == neutro_numeric:
                return 0
            elif valor_numeric <= neutro_numeric:
                return ((neutro_numeric - valor_numeric) / neutro_numeric) * peso
            else:
                return -((valor_numeric - neutro_numeric) / (bom_numeric - neutro_numeric)) * peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            bom_numeric = 1.0 if bom else 0.0
            neutro_numeric = 1.0 if neutro else 0.0
            if bom_numeric == neutro_numeric:
                return 0
            elif valor <= neutro_numeric:
                return ((neutro_numeric - valor) / neutro_numeric) * peso
            else:
                return -((valor - neutro_numeric) / (bom_numeric - neutro_numeric)) * peso
        elif bom < neutro:  # Quanto menor, melhor: BOM é o limite inferior ideal, NEUTRO o superior aceitável
            if valor <= bom:
                return peso  # Pontuação máxima se igual ou melhor que o ideal
            elif bom < valor <= neutro:
                return ((neutro - valor) / (neutro - bom)) * peso  # Pontuação diminui de peso para 0
            elif valor > neutro:
                return -((valor - neutro) / neutro) * peso # Pontuação negativa se pior que o aceitável
        elif bom > neutro: # Lógica para o caso onde o 'BOM' é um valor indesejavelmente alto (raro para "menor é melhor")
            if valor >= bom:
                return peso
            elif neutro <= valor < bom:
                return ((valor - neutro) / (bom - neutro)) * peso
            elif valor < neutro:
                return -((neutro - valor) / neutro) * peso
        else: # bom == neutro, já tratado no início
            return 0
    else:
        return 0
        return 0

# Aplicando a função de pontuação para cada atributo relevante
for col in montante_relacional.index:
    if tipos_dados[col] in ['number', 'boolean']:
        calculo_df[col + '_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao(x[col], valores_bom_df[col], valores_neutro_df[col], montante_relacional[col], proporcionalidade[col]),
            axis=1
        )

# Somando as pontuações para obter a pontuação total
colunas_pontuacao = [col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] in ['number', 'boolean']]
calculo_df['Pontuacao_Total'] = calculo_df[colunas_pontuacao].sum(axis=1)

# DataFrame para análise (incluindo BOM e NEUTRO com suas respectivas "pontuações")
df_analise = pd.concat([calculo_df[['Modelo', 'Pontuacao_Total']]], ignore_index=True)

# Adicionando uma coluna para colorir os pontos no gráfico
df_analise['Color'] = df_analise['Modelo'].apply(lambda x: 'Classificadores' if x in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if pd.notna(df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0]) and df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0] >= 0 else 'Pontuação Negativa'))

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',
    'Pontuação Positiva': 'Pontuação positiva',
    'Pontuação Negativa': 'Pontuação negativa'
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Ordenando os produtos pela pontuação total (mantendo BOM e NEUTRO no final para visualização)
df_analise_sorted = df_analise.sort_values(by='Pontuacao_Total', ascending=False, na_position='last').reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados (excluindo BOM e NEUTRO)
produtos_desempenho = calculo_df[~calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]
if not produtos_desempenho.empty:
    pontuacao_maxima = produtos_desempenho['Pontuacao_Total'].max()
    produtos_recomendados = produtos_desempenho[produtos_desempenho['Pontuacao_Total'] == pontuacao_maxima]

    # Exibindo todos os produtos recomendados com detalhes para leigos
    print("Produtos recomendados:")
    for _, produto in produtos_recomendados.iterrows():
        print(f"Produto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
        print("Pontos de vantagem e justificativas:")

        pontuacoes_detalhadas = []
        for col in montante_relacional.index:
            if tipos_dados[col] in ['number', 'boolean']:
                valor_produto = produto[col]
                bom_valor = valores_bom_df[col]
                neutro_valor = valores_neutro_df[col]
                pontuacao_atributo = calcular_pontuacao(
                    valor_produto,
                    bom_valor,
                    neutro_valor,
                    montante_relacional[col],
                    proporcionalidade[col]
                )
                if proporcionalidade[col] == 'proportional':
                    # ... (mesma lógica de justificativa para proporcional)
                    if isinstance(bom_valor, bool) and isinstance(neutro_valor, bool):
                        if valor_produto == bom_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto atende ao critério ideal ('BOM') para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto != neutro_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto não atende ao critério mínimo aceitável ('NEUTRO') para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                    elif pontuacao_atributo > 0:
                        justificativa = (
                            f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                            f"em comparação com o mínimo aceitável ('NEUTRO')."
                        )
                        pontuacoes_detalhadas.append(justificativa)
                    elif pontuacao_atributo < 0:
                        justificativa = (
                            f"O produto ficou abaixo no atributo '{col}' onde o desempenho neste atributo está {-pontuacao_atributo * (100 * (1 if isinstance(neutro_valor, bool) else neutro_valor)):.0f}% abaixo do mínimo aceitável ('NEUTRO')."
                        )
                        pontuacoes_detalhadas.append(justificativa)
                elif proporcionalidade[col] == 'i_proportional':
                    # ... (mesma lógica de justificativa para inversamente proporcional)
                    if isinstance(bom_valor, bool) and isinstance(neutro_valor, bool):
                        if valor_produto == neutro_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto atende ao critério ideal ('NEUTRO' para atributos inversamente proporcionais) para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto != bom_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto não atinge o nível menos desejado ('BOM' para atributos inversamente proporcionais) para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                    elif bom_valor > neutro_valor: # Ideal é menor
                        if valor_produto <= neutro_valor:
                            justificativa = f"O produto atingiu o ideal ('NEUTRO') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto > bom_valor:
                            justificativa = f"O produto está no nível menos desejado ('BOM') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo > 0:
                            justificativa = f"O produto está performando bem no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo < 0:
                            justificativa = f"O produto precisa melhorar no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                    elif bom_valor < neutro_valor: # Ideal é maior
                        if valor_produto >= neutro_valor:
                            justificativa = f"O produto atingiu o ideal ('NEUTRO') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto < bom_valor:
                            justificativa = f"O produto está no nível menos desejado ('BOM') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo > 0:
                            justificativa = f"O produto está performando bem no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo < 0:
                            justificativa = f"O produto precisa melhorar no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)

        for detalhe in pontuacoes_detalhadas:
            print(detalhe)
        print("\n")
else:
    print("Nenhum produto de desempenho encontrado para recomendação.")

# Plotando o gráfico
graph = px.bar(df_analise_sorted, x='Pontuacao_Total', y='Modelo', orientation='h',
             color='Color', text_auto=False, hover_name='Modelo',
             hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
             color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise_sorted) - 1,
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()

Produtos recomendados:
Produto: Apple com pontuação total de 1.51
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 90% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 21% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto atingiu o ideal ('NEUTRO') para o atributo 'Tamanho de tela' (inversamente proporcional).
O produto teve vantagem no atributo 'Bateria mAh' onde o desempenho neste atributo está 30% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto atende ao critério ideal ('BOM') para o atributo 'Bluetooth'.




### Teste Carro

In [74]:
import pandas as pd
import plotly.express as px
import numpy as np

# Leitura do arquivo CSV com a codificação UTF-8
teste_df = pd.read_csv('base_3.csv', encoding='UTF-8', sep=';')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
proporcionalidade = teste_df.iloc[2].drop('Modelo')

# Separando os DataFrames de 'BOM', 'NEUTRO' e os demais para cálculo
valores_bom_df = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro_df = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')
calculo_df = teste_df.drop([0, 1, 2], errors='ignore').reset_index(drop=True)

# Convertendo os tipos de dados conforme especificado
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom_df[col] = float(valores_bom_df[col])
        valores_neutro_df[col] = float(valores_neutro_df[col])
        calculo_df[col] = calculo_df[col].astype(float)
    elif tipos_dados[col] == 'boolean':
        valores_bom_df[col] = valores_bom_df[col] == 'TRUE'
        valores_neutro_df[col] = valores_neutro_df[col] == 'TRUE'
        calculo_df[col] = calculo_df[col].map({'TRUE': True, 'FALSE': False})

# Calculo da pontuação (inclui boolean e proporcionalidades)
def calcular_pontuacao(valor, bom, neutro, peso, proporcionalidade_tipo):
    if bom == neutro:
        return 0
    elif proporcionalidade_tipo == 'proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso  # Caso inesperado
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom == neutro:
            return 0
        elif valor >= neutro:
            return ((valor - neutro) / (bom - neutro)) * peso
        else:
            return -((neutro - valor) / neutro) * peso
    elif proporcionalidade_tipo == 'i_proportional':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif isinstance(bom, bool) and isinstance(neutro, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso
        elif bom < neutro:
            if valor <= bom:
                return peso
            elif bom < valor <= neutro:
                return ((neutro - valor) / (neutro - bom)) * peso
            elif valor > neutro:
                return -((valor - neutro) / neutro) * peso
        elif bom > neutro:
            if valor >= bom:
                return peso
            elif neutro <= valor < bom:
                return ((valor - neutro) / (bom - neutro)) * peso
            elif valor < neutro:
                return -((neutro - valor) / neutro) * peso
        else:
            return 0
    elif proporcionalidade_tipo == 'boolean':
        if isinstance(valor, bool):
            if valor == bom:
                return peso
            elif valor == neutro:
                return 0
            else:
                return -peso # Caso inesperado
        else:
            return 0 # Tratar se o tipo de dado não for booleano
    else:
        return 0

# Aplicando a função de pontuação para cada atributo relevante
for col in montante_relacional.index:
    if tipos_dados[col] in ['number', 'boolean']:
        calculo_df[col + '_pontuacao'] = calculo_df.apply(
            lambda x: calcular_pontuacao(x[col], valores_bom_df[col], valores_neutro_df[col], montante_relacional[col], proporcionalidade[col]),
            axis=1
        )

# Somando as pontuações para obter a pontuação total
colunas_pontuacao = [col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] in ['number', 'boolean']]
calculo_df['Pontuacao_Total'] = calculo_df[colunas_pontuacao].sum(axis=1)

# DataFrame para análise (incluindo BOM e NEUTRO com suas respectivas "pontuações")
df_analise = pd.concat([calculo_df[['Modelo', 'Pontuacao_Total']]], ignore_index=True)

# Adicionando uma coluna para colorir os pontos no gráfico
df_analise['Color'] = df_analise['Modelo'].apply(lambda x: 'Classificadores' if x in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if pd.notna(df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0]) and df_analise.loc[df_analise['Modelo'] == x, 'Pontuacao_Total'].iloc[0] >= 0 else 'Pontuação Negativa'))

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',
    'Pontuação Positiva': 'Pontuação positiva',
    'Pontuação Negativa': 'Pontuação negativa'
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Ordenando os produtos pela pontuação total (mantendo BOM e NEUTRO no final para visualização)
df_analise_sorted = df_analise.sort_values(by='Pontuacao_Total', ascending=False, na_position='last').reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados (excluindo BOM e NEUTRO)
produtos_desempenho = calculo_df[~calculo_df['Modelo'].isin(['BOM', 'NEUTRO'])]
if not produtos_desempenho.empty:
    pontuacao_maxima = produtos_desempenho['Pontuacao_Total'].max()
    produtos_recomendados = produtos_desempenho[produtos_desempenho['Pontuacao_Total'] == pontuacao_maxima]

    # Exibindo todos os produtos recomendados com detalhes para leigos
    print("Produtos recomendados:")
    for _, produto in produtos_recomendados.iterrows():
        print(f"Produto: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
        print("Pontos de vantagem e justificativas:")

        pontuacoes_detalhadas = []
        for col in montante_relacional.index:
            if tipos_dados[col] in ['number', 'boolean']:
                valor_produto = produto[col]
                bom_valor = valores_bom_df[col]
                neutro_valor = valores_neutro_df[col]
                pontuacao_atributo = calcular_pontuacao(
                    valor_produto,
                    bom_valor,
                    neutro_valor,
                    montante_relacional[col],
                    proporcionalidade[col]
                )
                if proporcionalidade[col] == 'proportional':
                    # ... (mesma lógica de justificativa para proporcional)
                    if isinstance(bom_valor, bool) and isinstance(neutro_valor, bool):
                        if valor_produto == bom_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto atende ao critério ideal ('BOM') para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto != neutro_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto não atende ao critério mínimo aceitável ('NEUTRO') para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                    elif pontuacao_atributo > 0:
                        justificativa = (
                            f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                            f"em comparação com o mínimo aceitável ('NEUTRO')."
                        )
                        pontuacoes_detalhadas.append(justificativa)
                    elif pontuacao_atributo < 0:
                        justificativa = (
                            f"O produto ficou abaixo no atributo '{col}' onde o desempenho neste atributo está {-pontuacao_atributo * (100 * (1 if isinstance(neutro_valor, bool) else neutro_valor)):.0f}% abaixo do mínimo aceitável ('NEUTRO')."
                        )
                        pontuacoes_detalhadas.append(justificativa)
                elif proporcionalidade[col] == 'i_proportional':
                    # ... (mesma lógica de justificativa para inversamente proporcional)
                    if isinstance(bom_valor, bool) and isinstance(neutro_valor, bool):
                        if valor_produto == neutro_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto atende ao critério ideal ('NEUTRO' para atributos inversamente proporcionais) para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto != bom_valor and bom_valor != neutro_valor:
                            justificativa = f"O produto não atinge o nível menos desejado ('BOM' para atributos inversamente proporcionais) para o atributo '{col}'."
                            pontuacoes_detalhadas.append(justificativa)
                    elif bom_valor > neutro_valor: # Ideal é menor
                        if valor_produto <= neutro_valor:
                            justificativa = f"O produto atingiu o ideal ('NEUTRO') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto > bom_valor:
                            justificativa = f"O produto está no nível menos desejado ('BOM') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo > 0:
                            justificativa = f"O produto está performando bem no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo < 0:
                            justificativa = f"O produto precisa melhorar no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                    elif bom_valor < neutro_valor: # Ideal é maior
                        if valor_produto >= neutro_valor:
                            justificativa = f"O produto atingiu o ideal ('NEUTRO') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif valor_produto < bom_valor:
                            justificativa = f"O produto está no nível menos desejado ('BOM') para o atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo > 0:
                            justificativa = f"O produto está performando bem no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)
                        elif pontuacao_atributo < 0:
                            justificativa = f"O produto precisa melhorar no atributo '{col}' (inversamente proporcional)."
                            pontuacoes_detalhadas.append(justificativa)

        for detalhe in pontuacoes_detalhadas:
            print(detalhe)
        print("\n")
else:
    print("Nenhum produto de desempenho encontrado para recomendação.")

# Plotando o gráfico
graph = px.bar(df_analise_sorted, x='Pontuacao_Total', y='Modelo', orientation='h',
             color='Color', text_auto=False, hover_name='Modelo',
             hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
             color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': df_analise_sorted['Modelo'].tolist()[::-1]})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise_sorted) - 1,
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()

Produtos recomendados:
Produto: Honda City com pontuação total de 0.53
Pontos de vantagem e justificativas:
O produto atingiu o ideal ('NEUTRO') para o atributo 'Preço (R$)' (inversamente proporcional).
O produto teve vantagem no atributo 'Consumo (km/l)' onde o desempenho neste atributo está 4% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Potência (cv)' onde o desempenho neste atributo está 12% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Porta-malas (litros)' onde o desempenho neste atributo está 10% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Segurança (estrelas)' onde o desempenho neste atributo está 20% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').




In [75]:
teste_df.head()

,Modelo,Preço (R$),Consumo (km/l),Potência (cv),Porta-malas (litros),Segurança (estrelas),Ar Condicionado,Câmbio Automático,Cor
0,NaN,0.3,0.1,0.1,0.05,0.2,0.1,0.1,0.05
1,NaN,number,number,number,number,number,boolean,boolean,string
2,NaN,i_proportional,proportional,proportional,proportional,proportional,boolean,boolean,NaN
3,BOM,60000,15,120,400,5,TRUE,TRUE,Branco
4,NEUTRO,85000,10,90,300,3,TRUE,FALSE,Prata


In [77]:
calculo_df.head(15)

,Modelo,Preço (R$),Consumo (km/l),Potência (cv),Porta-malas (litros),Segurança (estrelas),Ar Condicionado,Câmbio Automático,Cor,Preço (R$)_pontuacao,Consumo (km/l)_pontuacao,Potência (cv)_pontuacao,Porta-malas (litros)_pontuacao,Segurança (estrelas)_pontuacao,Ar Condicionado_pontuacao,Câmbio Automático_pontuacao,Pontuacao_Total
0,BOM,60000.0,15.0,120.0,400.0,5.0,True,True,Branco,0.300000,0.100,0.100000,0.050000,0.2,0,0.1,0.850000
1,NEUTRO,85000.0,10.0,90.0,300.0,3.0,True,False,Prata,0.000000,0.000,0.000000,0.000000,0.0,0,0.0,0.000000
2,Fiat Argo,75000.0,13.5,109.0,350.0,4.0,True,False,Vermelho,0.120000,0.070,0.063333,0.025000,0.1,0,0.0,0.378333
3,Volkswagen Polo,82000.0,12.8,116.0,320.0,5.0,True,False,Azul,0.036000,0.056,0.086667,0.010000,0.2,0,0.0,0.388667
4,Chevrolet Onix,78000.0,14.2,106.0,305.0,4.0,True,False,Preto,0.084000,0.084,0.053333,0.002500,0.1,0,0.0,0.323833
5,Hyundai HB20,72000.0,13.8,101.0,300.0,4.0,True,False,Branco,0.156000,0.076,0.036667,0.000000,0.1,0,0.0,0.368667
6,Renault Kwid,65000.0,15.5,70.0,290.0,3.0,True,False,Prata,0.240000,0.110,-0.022222,-0.001667,0.0,0,0.0,0.326111
7,Toyota Yaris,88000.0,12.5,110.0,310.0,4.0,True,True,Cinza,-0.010588,0.050,0.066667,0.005000,0.1,0,0.1,0.311078
8,Honda City,92000.0,11.8,126.0,500.0,5.0,True,True,Azul,-0.024706,0.036,0.120000,0.100000,0.2,0,0.1,0.531294
9,Peugeot 208,80000.0,13.0,113.0,315.0,4.0,True,False,Vermelho,0.060000,0.060,0.076667,0.007500,0.1,0,0.0,0.304167


## **CÉLULA 2 - DADOS NÃO NUMÉRICOS**

In [ ]:
df = pd.DataFrame()

# Situação/objeto exemplo: compra de um celular
# Começar considerando cores como categoria a ser analisada
# Considerar Vermelho como Muito desejável, Preto como Neutro e Verde como Nada desejável

# A ideia é a criação de diversos métodos de análise e com gráficos a serem analisados
# No caso, cada cor receberia um slider para que o usuário selecione a sua importância
# Considerando valores entre os intervalos dados (trabalhando na base de 0 até 100, logo, números decimais)
# Onde 0 é o mínimo desejável e 1 é o máximo desejável

'''
  Instruções de uso:
    - Com o dataframe já carregado da CÉLULA 1, basta alterar os dados de teste
    (placeholder) aqui presentes e executar o código.
    DICA: Crie cópias da célula, assim execute o código uma vez para cada dado
    qualitativo que desejar comparar.
'''

# Altere aqui! - Início
df['Cores'] = ['Vermelho', 'Verde', 'Branco', 'Preto', 'BOM', 'NEUTRO']
df['Valor Cores'] = [90, 10, 40, 50, 80, 50]
# Altere aqui! 0 Final

df['Tam_scatter'] = [1]*len(df)
symbols = ['diamond-wide']*len(df)

df['Valor X'] = [0]*len(df)

val_neutro = df['Valor Cores'][df.index[df['Cores'] == 'NEUTRO'][0]]
val_bom = df['Valor Cores'][df.index[df['Cores'] == 'BOM'][0]]

df['Definicao'] = pd.Series(dtype='str')
for i in range(len(df)):
    if df['Cores'][i] == 'NEUTRO' or df['Cores'][i] == 'BOM':
        df.loc[i, 'Definicao'] = 'Classificador'
    else:
        df['Definicao'] = 'Desempenho'
# Diversas cores para daltônicos

cores1 = ['red', 'blue']
cores2 = ['pink', 'green']
cores3 = ['orange', 'purple']
cores4 = ['orange', 'gray']

graph = px.scatter(df, x='Valor X', y='Valor Cores', title='Comparação com Indicadores de Desempenho e Classificadores',
                   color='Cores', symbol_sequence=symbols,
                   hover_data={'Cores':False, 'Valor Cores':False, 'Valor X':False,
                               'Valor Referencial': (df['Cores'])})

graph.update_traces(marker={'size': 20})
graph.update_xaxes(fixedrange=True, title=None)
graph.update_yaxes(fixedrange=True, title=None)

for i in range(len(df)):
    if df['Definicao'][i] == 'Classificador' and df['Cores'][i] == 'NEUTRO':
        graph.add_hline(y=df['Valor Cores'][i], line_color=cores3[1], line_width=3, x0=0.3, x1=0.7)
    elif df['Definicao'][i] == 'Classificador' and df['Cores'][i] == 'BOM':
        graph.add_hline(y=df['Valor Cores'][i], line_color=cores3[0], line_width=3, x0=0.3, x1=0.7)


graph.update_xaxes(showticklabels=False, showgrid=False)
graph.update_yaxes(showticklabels=False, showgrid=False)

graph.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=1,
    title_text='Variáveis e Cores'
))

graph.add_vline(x=0, y0=0, y1=1)
graph.add_annotation(
    x=-0.2,
    y=110,
    xref='x',
    yref='y',
    text=f'Linha Neutra<br>(Mínimo Aceitável)',
    showarrow=False,
    font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=cores3[1],
        opacity=1
)
graph.add_annotation(
    x=0.2,
    y=110,
    xref='x',
    yref='y',
    text=f'Linha Bom<br>(Muito Desejável)',
    showarrow=False,
    font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=cores3[0],
        opacity=1
)
graph.show()